In [1]:
import numpy

# Spherical coordinates

Latitude, $\phi$, and longitude, $\lambda$, determine position on the surface of a sphere of radius, $r$.

The Cartesian coordinates $(x,y,z)$ of position $(\lambda,\phi)$ are

\begin{align}
x &= r \cos{\lambda} \cos{\phi} \\
y &= r \sin{\lambda} \cos{\phi} \\
z &= r \sin{\phi}
\end{align}

The spherical coordinate $(\lambda,\phi)$ in terms Cartesian coordinates $(x,y,z)$ are

\begin{align}
\lambda &= \tan^{-1}{ \frac{y}{x} } \\
\phi &= \sin^{-1}{ \frac{z}{r} }
\end{align}

The line element is given by

\begin{align}
ds^2 &= h_\lambda^2 d\lambda^2 + h_\phi^2 d\phi^2
\end{align}

where the scale factors are

\begin{align}
h_\lambda &= \sqrt{
\left( \frac{\partial x}{\partial \lambda} \right)^2
+ \left( \frac{\partial y}{\partial \lambda} \right)^2
+ \left( \frac{\partial z}{\partial \lambda} \right)^2 }
\\
h_\phi &= \sqrt{
\left( \frac{\partial x}{\partial \phi} \right)^2
+ \left( \frac{\partial y}{\partial \phi} \right)^2
+ \left( \frac{\partial z}{\partial \phi} \right)^2 }
\end{align}

The scale factors for the spherical projection simplify to

\begin{align}
h_\lambda &= r \cos{\phi} \\
h_\phi &= r
\end{align}
so the line element is
$$
d{\mathbf s} = \hat{\boldsymbol \lambda} r \cos{\phi} d\lambda + \hat{\boldsymbol \phi} r d\phi
$$
where the unit vectors are

\begin{align}
\hat{\boldsymbol \lambda} &=
\left( \begin{array}{c}
- \sin{\lambda} \\ \cos{\lambda} \\ 0
\end{array} \right)
\\
\hat{\boldsymbol \phi} &=
\left( \begin{array}{c}
\cos{\lambda} \sin{\phi} \\ \sin{\lambda} \sin{\phi} \\ \cos{\phi}
\end{array} \right)
\end{align}

The area element is
$$
d{\mathbf a} = r^2 \cos{\phi} d\lambda d\phi
$$

A line segment of constant longitude has length $\Delta s = \int h_\phi d\phi = r \Delta \phi$ where $\Delta \phi$ is the change in latitude for the segment.

A line segment of constant latitude has length $\Delta s = \int h_\lambda d\lambda = ( r \cos{\phi} ) \Delta \lambda$ where $\Delta \lambda$ is the change in longitude for the segment.

## A spherical grid

A spherical grid is a mesh made from lines of constant longitude uniformly spaced by $\Delta \lambda$ and lines of constant latitude spaced by $\Delta \phi$.

The mesh parameters are the starting longitude, $\lambda_0$, starting latitude, $\phi_0$ and the spacings $\Delta \lambda$ and $\Delta \phi$.

For $ni \times nj$ cells there are $(ni+1)(nj+1)$ nodes.

Positions of the nodes are

\begin{align}
\lambda_i = \lambda_0 + i \Delta \lambda \;\;\; i = 0, 1, \ldots, ni \\
\phi_j = \phi_0 + j \Delta \phi \;\;\; j = 0, 1, \ldots, nj
\end{align}

The mesh element lengths are

\begin{align}
\Delta y_{i,j+1/2} = r \left( \phi_{j+1} - \phi_j \right) = r \Delta \phi
& \;\; \forall \;\; i = 0, 1, \ldots, ni \;\; \textrm{and} \;\; j = 0, 1, \ldots, nj-1 \\
\Delta x_{i+1/2,j} = r \cos{\phi_j} \left( \lambda_{i+1} - \lambda_i \right) = r \cos{\phi_j} \Delta \lambda
& \;\; \forall \;\; i = 0, 1, \ldots, ni-1 \;\; \textrm{and} \;\; j = 0, 1, \ldots, nj
\end{align}

The cell areas are
$$
A_{i+1/2,j+1/2} = r^2 \left( \sin{\phi_{j+1}} - \sin{\phi_j} \right) \Delta \phi \Delta \lambda
\;\; \forall \;\; i = 0, 1, \ldots, ni-1 \;\; \textrm{and} \;\; j = 0, 1, \ldots, nj-1
$$

## Numerical approximation of the scale factors

Since $\lambda$,$\phi$ are orthogonal coordinates we can measure the scale factors

\begin{align}
h_\lambda &= \left. \frac{\partial s}{\partial \lambda} \right|_\phi
\\
h_\phi &= \left. \frac{\partial s}{\partial \phi} \right|_\lambda
\end{align}

The derivatives can be approximated numerically. For example, using second order centered differences

\begin{align}
h_\lambda &= \left. \frac{\partial s}{\partial \lambda} \right|_\phi
\\
&= \left. \frac{\partial s}{\partial i} \right|_\phi \left. \frac{\partial i}{\partial \lambda} \right|_\phi
\\
&\approx \frac{ \left( \frac{ s(i+\epsilon,j) - s(i-\epsilon,j) }{ 2\epsilon } + O(\epsilon^2) \right)
}{ \left( \frac{ \lambda(i+\epsilon,j) - \lambda(i-\epsilon,j) }{ 2\epsilon } + O(\epsilon^2) \right) }
\\
&\approx \frac{ \left( s(i+\epsilon,j) - s(i-\epsilon,j) \right)
}{ \left( \lambda(i+\epsilon,j) - \lambda(i-\epsilon,j) \right) } + O(\epsilon^2)
\\
&\approx \frac{
\left( s(i+\epsilon,j) - s(i-\epsilon,j) \right)
}{ 2 \epsilon \Delta \lambda } + O(\epsilon^2)
\end{align}

For small $\epsilon$ the distance $s(i+\epsilon,j) - s(i-\epsilon,j)$ can be approximated by the great arc distance. Higher order differences can be substituted for accuracy.

In [2]:
class SG:
    def __init__(self, lon0, lat0, dlon, dlat, r=1):
        """A spherical grid starting at (lon0,lat0) with spacing (dlon,dlat)
        given in degrees. r=1 by default."""
        self.r = r
        self.lon0, self.lat0 = lon0, lat0
        self.dlon, self.dlat = dlon, dlat
        deg2rad = numpy.pi / 180.
        self.dlam, self.dphi = deg2rad * dlon, deg2rad * dlat
    def lon(self, j, i):
        """Returns longitude of nodes in degrees"""
        return self.lon0 + self.dlon * numpy.array(i)
    def lat(self, j, i):
        """Returns latitude of nodes in degrees"""
        return self.lat0 + self.dlat * numpy.array(j)
    def lam(self, j, i):
        """Returns longitude of nodes in radians"""
        deg2rad = numpy.pi / 180.
        return self.lon(j, i) * deg2rad
    def phi(self, j, i):
        """Returns latitude of nodes in radians"""
        deg2rad = numpy.pi / 180.
        return self.lat(j, i) * deg2rad
    def dy(self, j, i):
        """Returns length of latitude segments"""
        phi = self.phi(j, i)
        return self.r * ( phi[1:,:] - phi[:-1,:] )
    def dx(self, j, i):
        """Returns length of longitude segments"""
        phi, lam = self.phi(j, i), self.lam(j, i)
        phi = 0.5 * ( phi[:,1:] + phi[:,:-1] )
        dlam = lam[:,1:] - lam[:,:-1]
        return self.r * numpy.cos(phi) * dlam
    def area(self, j, i):
        """Returns areas of cells"""
        phi, lam = self.phi(j, i), self.lam(j, i)
        phi = 0.5 * ( phi[:,1:] + phi[:,:-1] )
        dphi = phi[1:,:] - phi[:-1,:]
        lam = 0.5 * ( lam[1:,:] + lam[:-1,:] )
        dlam = lam[:,1:] - lam[:,:-1]
        return self.r**2 * ( numpy.sin(phi[1:,:]) - numpy.sin(phi[:-1,:]) ) * dlam
    def hi(self, j, i):
        """Returns scale factor h_lambda"""
        phi = self.phi(j,i)
        return self.r * numpy.cos( phi )
    def hj(self, j, i):
        """Returns scale factor h_phi"""
        phi = self.phi(j,i)
        return self.r + 0 * phi
    def great_arc_distance(self, j0, i0, j1, i1):
        """Returns great arc distance between nodes (j0,i0) and (j1,i1)"""
        # https://en.wikipedia.org/wiki/Great-circle_distance
        phi0, lam0 = self.phi(j0,i0), self.lam(j0,i0)
        phi1, lam1 = self.phi(j1,i1), self.lam(j1,i1)
        dphi, dlam = phi1 - phi0, lam1 - lam0
        # Haversine formula
        d = numpy.sin( 0.5 * dphi)**2 + numpy.sin( 0.5 * dlam)**2 * numpy.cos(phi0) * numpy.cos(phi1)
        return self.r * 2. * numpy.arcsin( numpy.sqrt( d ) )
    def numerical_hi(self, j, i, eps, order=6):
        """Returns a numerical approximation to h_lambda"""
        reps = 1. / ( self.dlam * eps )
        ds2 = self.great_arc_distance(j, i+eps, j, i-eps)
        if order == 2: return 0.5 * ds2 * reps
        ds4 = self.great_arc_distance(j, i+2.*eps, j, i-2.*eps)
        if order == 4: return ( 8. * ds2 - ds4 ) * (1./12.) * reps
        ds6 = self.great_arc_distance(j, i+3.*eps, j, i-3.*eps)
        if order == 6: return ( 45. * ds2 - 9. * ds4 + ds6 ) * (1./60.) * reps
        raise Exception('order not coded')
    def numerical_hj(self, j, i, eps, order=6):
        """Returns a numerical approximation to h_phi"""
        reps = 1. / ( self.dphi * eps )
        ds2 = self.great_arc_distance(j+eps, i, j-eps, i)
        if order == 2: return 0.5 * ds2 * reps
        ds4 = self.great_arc_distance(j+2.*eps, i, j-2.*eps, i)
        if order == 4: return ( 8. * ds2 - ds4 ) * (1./12.) * reps
        ds6 = self.great_arc_distance(j+3.*eps, i, j-3.*eps, i)
        if order == 6: return ( 45. * ds2 - 9. * ds4 + ds6 ) * (1./60.) * reps
        raise Exception('order not coded')

In [3]:
g = SG(0, 0, 60, 45)
i,j = numpy.meshgrid( numpy.arange(7), numpy.arange(5) ) # 6 x 4 cells
print( 'lon =', g.lon(j,i) )
print( 'lat =', g.lat(j,i) )
print( 'dx / pi =', g.dx(j,i) / numpy.pi)
print( 'dy / pi =', g.dy(j,i) / numpy.pi )
print( 'area / pi =', g.area(j,i) / numpy.pi )
print( 'hi =', g.hi(j,i) )
print( 'hj =', g.hj(j,i) )

lon = [[  0  60 120 180 240 300 360]
 [  0  60 120 180 240 300 360]
 [  0  60 120 180 240 300 360]
 [  0  60 120 180 240 300 360]
 [  0  60 120 180 240 300 360]]
lat = [[  0   0   0   0   0   0   0]
 [ 45  45  45  45  45  45  45]
 [ 90  90  90  90  90  90  90]
 [135 135 135 135 135 135 135]
 [180 180 180 180 180 180 180]]
dx / pi = [[ 3.33333333e-01  3.33333333e-01  3.33333333e-01  3.33333333e-01
   3.33333333e-01  3.33333333e-01]
 [ 2.35702260e-01  2.35702260e-01  2.35702260e-01  2.35702260e-01
   2.35702260e-01  2.35702260e-01]
 [ 2.04107800e-17  2.04107800e-17  2.04107800e-17  2.04107800e-17
   2.04107800e-17  2.04107800e-17]
 [-2.35702260e-01 -2.35702260e-01 -2.35702260e-01 -2.35702260e-01
  -2.35702260e-01 -2.35702260e-01]
 [-3.33333333e-01 -3.33333333e-01 -3.33333333e-01 -3.33333333e-01
  -3.33333333e-01 -3.33333333e-01]]
dy / pi = [[0.25 0.25 0.25 0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25 0.25 0.25 0.25

In [4]:
print( 'Total area / 4 pi =', g.area(j,i).sum() / (4*numpy.pi) )
print( 'Lower hemisphere area / 2 pi =',g.area(j,i)[:2,:].sum() / (2*numpy.pi) )
print( 'Western hemisphere area / 2 pi =',g.area(j,i)[:,:3].sum() / (2*numpy.pi) )
print( 'Pole-to-pole distance / pi =',g.dy(j,i)[:,0].sum() / numpy.pi )
print( 'Length of equator / 2 pi =',g.dx(j,i)[2,:].sum() / (2*numpy.pi) )

Total area / 4 pi = 3.533949646070574e-17
Lower hemisphere area / 2 pi = 0.9999999999999999
Western hemisphere area / 2 pi = 1.766974823035287e-17
Pole-to-pole distance / pi = 1.0
Length of equator / 2 pi = 6.123233995736766e-17


In [5]:
# Check great arc distance == dy
print( g.great_arc_distance(j[:-1],i[:-1],j[1:],i[1:]) / g.dy(j,i) )
# Check great arc distance <= dx
print( g.great_arc_distance(j[:,:-1],i[:,:-1],j[:,1:],i[:,1:]) / g.dx(j,i) )

[[1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]]
[[ 1.          1.          1.          1.          1.          1.        ]
 [ 0.97603415  0.97603415  0.97603415  0.97603415  0.97603415  0.97603415]
 [ 0.95492966  0.95492966  0.95492966  0.95492966  0.95492966  0.95492966]
 [-0.97603415 -0.97603415 -0.97603415 -0.97603415 -0.97603415 -0.97603415]
 [-1.         -1.         -1.         -1.         -1.         -1.        ]]


In [6]:
# Compare numerical scale factor hi to true scale factor hi
g.numerical_hi(j,i,1e-2,order=6) / g.hi(j,i)

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [-1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1.]])

In [7]:
# Compare numerical scale factor hj to true scale factor hj
g.numerical_hj(j,i,1e-2,order=6) / g.hj(j,i)

array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]])